# Modelo de Recomendación Item-Item:


El modelo debe establecer una relación ítem-ítem, donde la recomendación de elementos similares se basa en la medida de similitud entre un ítem dado y el resto. En este contexto, el modelo toma como entrada un juego específico y genera como salida una lista de juegos recomendados. Para lograr esto, se recomienda aplicar la métrica de similitud del coseno.

### 1. Importar las librerías necesarias para la creación del modelo y descarga de datos:

In [1]:
import pandas as pd                                                 # Para abrir y editar los archivos 
import numpy as np                                                  # Para realizar cálculos

#----------------------------------------------------------------------------------------------------------------------------------

from sklearn.feature_extraction.text import CountVectorizer         # Para convertir las características analizadas en matrices
from sklearn.metrics.pairwise import cosine_similarity              # Para calcular la similitud de coseno de la matriz


### 2. Abrir y arreglar los datos para que no hayan registros nulos:

In [2]:
# Abrir arcchivo "Genres_Items"
df_genres_expanded= pd.read_csv('Clean_Data/Genres_Items.csv')

# Eliminar registros nulos en 'id' o 'genre'
df_genres_expanded = df_genres_expanded.dropna(subset=['id', 'genres'])

df_genres_expanded.info()

<class 'pandas.core.frame.DataFrame'>
Index: 80408 entries, 0 to 80408
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      80408 non-null  float64
 1   genres  80408 non-null  object 
dtypes: float64(1), object(1)
memory usage: 1.8+ MB


### 3. Expandir la columna "genres":

In [3]:
data=df_genres_expanded

# Convertir las listas en cadenas separadas por comas
data['genres'] = df_genres_expanded['genres'].astype(str)

# Crear columnas para cada género con 1 y 0
data = pd.get_dummies(data['genres'], prefix='', prefix_sep='')

# Agregar la columna 'id' al nuevo DataFrame
data['id'] = df_genres_expanded['id'].abs()

# Agrupar por 'id' y tomar el máximo para combinar las filas duplicadas
data = data.groupby('id').max()

# Resetear el índice
data = data.reset_index()

In [4]:
data

,id,Action,Adventure,Casual,Early Access,Free to Play,Indie,RPG,Racing,Simulation,Sports,Strategy,VR,others
0,10.0,True,False,False,False,False,False,False,False,False,False,False,False,False
1,20.0,True,False,False,False,False,False,False,False,False,False,False,False,False
2,30.0,True,False,False,False,False,False,False,False,False,False,False,False,False
3,40.0,True,False,False,False,False,False,False,False,False,False,False,False,False
4,50.0,True,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32127,2028055.0,True,False,False,False,False,False,False,False,False,False,False,False,False
32128,2028056.0,False,False,False,False,False,False,False,False,False,False,True,False,False
32129,2028062.0,True,False,False,False,False,False,False,False,False,False,False,False,False
32130,2028103.0,True,True,False,False,False,False,False,False,False,False,False,False,False


4. Abrir el archivo SGames_CD, eliminar las columnas no necesarias y los datos vacíos:

In [5]:
# Abrir arcchivo "Genres_Items"
Model_Data = pd.read_csv('Clean_Data\\SGames_CD.csv', quotechar='"')

#Usar las columnas necesarias
Model_Data=Model_Data[['id','genres','publisher','release_date','specs','price','early_access','developer','release_year']]

# Eliminar registros con valores nulos en 'genres' y valores de 0 en 'release_year'

Model_Data = Model_Data[(Model_Data['release_year'] != 0) & (Model_Data['genres'].notnull())]

# Reemplazar valores nulos en 'genres' con 'desconocido'
Model_Data['genres'] = Model_Data['genres'].fillna('a stranger')

# Reemplazar valores nulos en 'developer' y 'publisher' con 'desconocido':
Model_Data['developer'] = Model_Data['developer'].fillna('a stranger')
Model_Data['publisher'] = Model_Data['publisher'].fillna('a stranger')

# Colocar 0 en los valores nulos de 'price'
Model_Data['price'] = Model_Data['price'].fillna(0)

# Eliminar registros con valores nulos en 'genres'
Model_Data = Model_Data.dropna(subset=['genres'])

# Eliminar la columna 'release_date' y 'genres'
Model_Data = Model_Data.drop(columns=['release_date','genres'])

# Colocar 'not specified' en los valores nulos de la columna 'specs'
Model_Data['specs'] = Model_Data['specs'].fillna('not specified')

# Verificar los cambios
Model_Data.head(-5)


,id,publisher,specs,price,early_access,developer,release_year
0,761140.0,Kotoshiro,['Single-player'],4.99,False,Kotoshiro,2018
1,643980.0,"Making Fun, Inc.","['Single-player', 'Multi-player', 'Online Mult...",0.00,False,Secret Level SRL,2018
2,670290.0,Poolians.com,"['Single-player', 'Multi-player', 'Online Mult...",0.00,False,Poolians.com,2017
3,767400.0,彼岸领域,['Single-player'],0.99,False,彼岸领域,2017
5,772540.0,Trickjump Games Ltd,"['Single-player', 'Steam Achievements']",3.99,False,Trickjump Games Ltd,2018
...,...,...,...,...,...,...,...
32122,761480.0,Phil Fortier,"['Single-player', 'Steam Achievements']",0.99,False,Phil Fortier,2018
32123,771810.0,Retro Army Limited,"['Single-player', 'Captions available']",0.00,False,Retro Army Limited,2018
32124,767590.0,OrtiGames/OrtiSoft,"['Single-player', 'Shared/Split Screen', 'Stea...",0.99,False,"Oscar Ortigueira López,OrtiGames/OrtiSoft",2018
32125,747320.0,INGAME,"['Single-player', 'Steam Achievements', 'Steam...",14.99,False,INGAME,2018


### 5. Unir las tablas para el modelo de recomendación:

In [6]:
df_model_data = pd.DataFrame(Model_Data)

# Merge (join) de los DataFrames por la columna 'id'
result = pd.merge(df_model_data, data, on='id')

# Eliminar la columna 'id' del DataFrame resultante
#result = result.drop('id', axis=1)

# Mostrar el resultado
result

,id,publisher,specs,price,early_access,developer,release_year,Action,Adventure,Casual,Early Access,Free to Play,Indie,RPG,Racing,Simulation,Sports,Strategy,VR,others
0,761140.0,Kotoshiro,['Single-player'],4.99,False,Kotoshiro,2018,True,False,True,False,False,True,False,False,True,False,True,False,False
1,643980.0,"Making Fun, Inc.","['Single-player', 'Multi-player', 'Online Mult...",0.00,False,Secret Level SRL,2018,False,False,False,False,True,True,True,False,False,False,True,False,False
2,670290.0,Poolians.com,"['Single-player', 'Multi-player', 'Online Mult...",0.00,False,Poolians.com,2017,False,False,True,False,True,True,False,False,True,True,False,False,False
3,767400.0,彼岸领域,['Single-player'],0.99,False,彼岸领域,2017,True,True,True,False,False,False,False,False,False,False,False,False,False
4,772540.0,Trickjump Games Ltd,"['Single-player', 'Steam Achievements']",3.99,False,Trickjump Games Ltd,2018,True,True,False,False,False,False,False,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29639,745400.0,Bidoniera Games,"['Single-player', 'Steam Achievements', 'Steam...",1.99,False,Bidoniera Games,2018,True,True,True,False,False,True,False,False,False,False,False,False,False
29640,773640.0,Ghost_RUS Games,"['Single-player', 'Steam Achievements']",1.99,False,"Nikita ""Ghost_RUS""",2018,False,False,True,False,False,True,False,False,True,False,True,False,False
29641,733530.0,Sacada,"['Single-player', 'Steam Achievements', 'Steam...",4.99,False,Sacada,2018,False,False,True,False,False,True,False,False,False,False,True,False,False
29642,610660.0,Laush Studio,"['Single-player', 'Steam Achievements', 'Steam...",1.99,False,Laush Dmitriy Sergeevich,2018,False,False,False,False,False,True,False,True,True,False,False,False,False


### 6. Convertir "specs" en una cadena sin carácteres, ni separadores para facilitar la lectura del modelo:

In [7]:
Model_Data=result

# Eliminar comillas, corchetes, comas y reemplazar guiones por espacios en 'specs'
Model_Data['specs'] = Model_Data['specs'].astype(str).str.replace('[', '').str.replace(']', '').str.replace("'", "").str.replace(",", "").str.replace("-", " ")

# Verificar los cambios
Model_Data.head()

,id,publisher,specs,price,early_access,developer,release_year,Action,Adventure,Casual,Early Access,Free to Play,Indie,RPG,Racing,Simulation,Sports,Strategy,VR,others
0,761140.0,Kotoshiro,Single player,4.99,False,Kotoshiro,2018,True,False,True,False,False,True,False,False,True,False,True,False,False
1,643980.0,"Making Fun, Inc.",Single player Multi player Online Multi Player...,0.00,False,Secret Level SRL,2018,False,False,False,False,True,True,True,False,False,False,True,False,False
2,670290.0,Poolians.com,Single player Multi player Online Multi Player...,0.00,False,Poolians.com,2017,False,False,True,False,True,True,False,False,True,True,False,False,False
3,767400.0,彼岸领域,Single player,0.99,False,彼岸领域,2017,True,True,True,False,False,False,False,False,False,False,False,False,False
4,772540.0,Trickjump Games Ltd,Single player Steam Achievements,3.99,False,Trickjump Games Ltd,2018,True,True,False,False,False,False,False,False,True,False,False,False,False


In [8]:
Model_Data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29644 entries, 0 to 29643
Data columns (total 20 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            29644 non-null  float64
 1   publisher     29644 non-null  object 
 2   specs         29644 non-null  object 
 3   price         29644 non-null  float64
 4   early_access  29644 non-null  bool   
 5   developer     29644 non-null  object 
 6   release_year  29644 non-null  int64  
 7   Action        29644 non-null  bool   
 8   Adventure     29644 non-null  bool   
 9   Casual        29644 non-null  bool   
 10  Early Access  29644 non-null  bool   
 11  Free to Play  29644 non-null  bool   
 12  Indie         29644 non-null  bool   
 13  RPG           29644 non-null  bool   
 14  Racing        29644 non-null  bool   
 15  Simulation    29644 non-null  bool   
 16  Sports        29644 non-null  bool   
 17  Strategy      29644 non-null  bool   
 18  VR            29644 non-nu

### 7. Desarrollo del modelo similitud del coseno (item-item):

In [9]:
# Seleccionar solo las columnas numéricas para el cálculo de similitud del coseno
numeric_columns = ['price', 'release_year']

# Normalizar las columnas numéricas
normalized_data = Model_Data[numeric_columns]
normalized_data = (normalized_data - normalized_data.mean()) / (normalized_data.max() - normalized_data.min())

# Concatenar las columnas normalizadas con las columnas categóricas
normalized_model_data = pd.concat([Model_Data[['id', 'publisher', 'specs', 'early_access', 'developer']], normalized_data], axis=1)

# Calcular la similitud del coseno
cosine_sim = cosine_similarity(normalized_data)

### 8. Creación de fórmula para que reciba un "id" de video juego y devuelva los 5 más parecidos a el, para el modelo de recomendación:

In [10]:
# Función para obtener las recomendaciones
def get_recommendations(game_id, n=5):
    # Obtener la fila correspondiente al juego seleccionado
    selected_game_row = normalized_model_data[normalized_model_data['id'] == game_id]

    # Obtener las similitudes del coseno con respecto al juego seleccionado
    similarities = cosine_sim[selected_game_row.index[0]]

    # Obtener los índices de los juegos más similares (excluyendo el juego seleccionado)
    similar_indices = similarities.argsort()[:-1][::-1][:n]

    # Obtener los ID de los juegos más similares
    recommended_ids = normalized_model_data.iloc[similar_indices]['id'].tolist()

    return recommended_ids


In [11]:
# Ejemplo de uso: Obtener las 5 recomendaciones para el juego con ID 1
recommended_games = get_recommendations(772540, n=5)
print(recommended_games)

[768880.0, 772450.0, 767010.0, 777930.0, 775640.0]


### 9. Iterar en la base de datos, para obtener para cada id los juesgos recomendados:

In [12]:
dic_similitud = {}

for i in Model_Data['id']:
    try:
        value = get_recommendations(i)
        if value == i:
            continue
        else:
            dic_similitud[i] = value
    except Exception as e:
        print(f"Error para el ID {i}: {str(e)}")

### 10. Crear dataframe del diciconario obtenido:

In [13]:
# Crear listas para almacenar datos
ids = []
recommended_ids = []

# Iterar sobre el diccionario y crear las listas
for key, values in dic_similitud.items():
    ids.extend([key] * len(values))
    recommended_ids.extend(values)

# Crear un DataFrame a partir de las listas
similitud_df = pd.DataFrame({'id': ids, 'recommended_id': recommended_ids})

# Imprimir el DataFrame resultante
similitud_df

,id,recommended_id
0,761140.0,733530.0
1,761140.0,412250.0
2,761140.0,769600.0
3,761140.0,653960.0
4,761140.0,777910.0
...,...,...
148215,658870.0,632870.0
148216,658870.0,632930.0
148217,658870.0,736710.0
148218,658870.0,573720.0


In [14]:
# Agrupar por 'id' y concatenar 'recommended_id' en una lista
grouped_df = similitud_df.groupby('id')['recommended_id'].agg(list).reset_index()

grouped_df


,id,recommended_id
0,10.0,"[13240.0, 345240.0, 615770.0, 32760.0, 692660.0]"
1,20.0,"[251770.0, 292840.0, 253940.0, 253920.0, 50.0]"
2,30.0,"[277460.0, 39670.0, 30.0, 301670.0, 352130.0]"
3,40.0,"[695630.0, 130.0, 40.0, 4880.0, 289820.0]"
4,50.0,"[251770.0, 292840.0, 253940.0, 253920.0, 50.0]"
...,...,...
29639,901805.0,"[253650.0, 91800.0, 403090.0, 22450.0, 200130.0]"
29640,2028055.0,"[214643.0, 313660.0, 219640.0, 207610.0, 20363..."
29641,2028056.0,"[211340.0, 208460.0, 214250.0, 205810.0, 20975..."
29642,2028103.0,"[204760.0, 510320.0, 200510.0, 50300.0, 67370.0]"


### 11. Exportar datos para ser usados por FastApi:

11.1. Crear tabla auxiliar con los nombres de videojuegos:

In [15]:
names = pd.read_csv('Clean_Data\\SGames_CD.csv', quotechar='"')
names=names[['id','app_name']]

In [21]:
# Exportar el DataFrame a CSV:

grouped_df.to_parquet('Data_Queries/similitud_df.parquet', index=False)
print(f"DataFrame exportado a {'Data_Queries/similitud_df.parquet'}")

names.to_csv('Data_Queries/names.csv', index=False)
print(f"DataFrame exportado a {'Data_Queries/names.csv'}")

DataFrame exportado a Data_Queries/similitud_df.parquet
DataFrame exportado a Data_Queries/names.csv


In [23]:
similitud_df = pd.read_parquet('Data_Queries\similitud_df.parquet')
#df_genres_expanded= pd.read_csv('Clean_Data/Genres_Items.csv')
  
similitud_df

,id,recommended_id
0,10.0,"[13240.0, 345240.0, 615770.0, 32760.0, 692660.0]"
1,20.0,"[251770.0, 292840.0, 253940.0, 253920.0, 50.0]"
2,30.0,"[277460.0, 39670.0, 30.0, 301670.0, 352130.0]"
3,40.0,"[695630.0, 130.0, 40.0, 4880.0, 289820.0]"
4,50.0,"[251770.0, 292840.0, 253940.0, 253920.0, 50.0]"
...,...,...
29639,901805.0,"[253650.0, 91800.0, 403090.0, 22450.0, 200130.0]"
29640,2028055.0,"[214643.0, 313660.0, 219640.0, 207610.0, 20363..."
29641,2028056.0,"[211340.0, 208460.0, 214250.0, 205810.0, 20975..."
29642,2028103.0,"[204760.0, 510320.0, 200510.0, 50300.0, 67370.0]"


### 12. Crear Función para FastApi:

In [18]:
def obtener_nombres_recomendados(id_seleccionado):
    try:
        # Obtener el nombre del juego seleccionado
        nombre_seleccionado = names.loc[names['id'] == id_seleccionado, 'app_name'].iloc[0]

        # Obtener los IDs de los juegos recomendados
        juegos_recomendados_ids = similitud_df.loc[similitud_df['id'] == id_seleccionado, 'recommended_id'].iloc[0]

        # Obtener los nombres de los juegos recomendados
        nombres_recomendados = []
        for id_recomendado in juegos_recomendados_ids:
            nombre_recomendado = names.loc[names['id'] == id_recomendado, 'app_name'].iloc[0]
            nombres_recomendados.append(nombre_recomendado)

        # Crear y devolver el diccionario
        diccionario_resultado = {'nombre_seleccionado': nombre_seleccionado, 'nombres_recomendados': nombres_recomendados}
        return diccionario_resultado

    except IndexError:
        print(f"No se encontró nombre para el ID seleccionado {id_seleccionado}")

    except KeyError:
        print(f"No se encontraron recomendaciones para el ID seleccionado {id_seleccionado}")

    except Exception as e:
        print(f"Error inesperado: {str(e)}")

12.1. Probar Función:

In [19]:
# Ejemplo de uso
id_seleccionado = 670290  # Ingresa el ID deseado
resultado = obtener_nombres_recomendados(id_seleccionado)
print(resultado)

{'nombre_seleccionado': 'Real Pool 3D - Poolians', 'nombres_recomendados': ['Solarium', '2 Ninjas 1 Cup - Soundtrack', 'A Step Into Darkness', '摸金侠', 'In The Dark']}
